***Vorlesung 'Syntax natürlicher Sprachen'***

--- 
# Intro: Syntaktische Konstruktionen und Komplexität

In [56]:
import nltk
from nltk.tree import Tree
from nltk import DependencyGraph

from itertools import chain

from spacy import displacy


def transform_nr_conll(sent_nr):
    sent_list = []
    for line in list(filter(None, sent_nr.split("\n"))):
        line_list = line.split()
        line_list.pop(0)
        line_list.insert(1,"_")
        sent_list.append(" ".join([i for i in line_list[0:]]))

    return "\n".join([i for i in sent_list[0:]])


def _tree_labeled(self, i):
        node = self.get_by_address(i)
        word = node["word"]
        rel = node["rel"]        
        deps = sorted(chain.from_iterable(node["deps"].values()))

        if deps:
            return Tree(word+'('+rel+')', [self._tree_labeled(dep) for dep in deps])
        else:
            return word+'('+rel+')'
        
def tree_labeled(self):
        node = self.root

        word = node["word"]
        rel = node["rel"]
        deps = sorted(chain.from_iterable(node["deps"].values()))
        return Tree(word+'('+rel+')', [self._tree_labeled(dep) for dep in deps])

DependencyGraph._tree_labeled = _tree_labeled
DependencyGraph.tree_labeled = tree_labeled


def displacy_dep_input(sent):
    deps = []
    for dep in sent.split('\n'):
        deps.append(dep.split())

    deps = [x for x in deps if x]

    ex = []
    word_list = []
    arc_list = []

    for index, dep in enumerate(deps):
        word_list.append({"text": dep[0], "tag": ""})
        line = index+1
        head = int(dep[2])
        label = dep[3]
        if head>line:
            start = index
            end = head-1
            direction = "left"
        else:
            start = head-1
            end = index  
            direction = "right"
        if(label.lower() != "root"):
            arc_list.append({"start": start, "end": end, "label": label, "dir": direction})

    ex.append({
        "words": word_list,
        "arcs": arc_list
    })    

    return ex



---
 
 
## Verarbeitung syntaktischer Konstruktionen

#### Gegeben sei folgende Liste von Sätzen:

In [41]:
constructions = [
"dass wir Max helfen das Haus zu streichen",
"dass wir Max das Haus streichen helfen",
"*dass wir Max das Haus helfen streichen"   # (im Deutschen ungrammatisch)
]

for (i, item) in enumerate(constructions):
    print(i, item)    

0 dass wir Max helfen das Haus zu streichen
1 dass wir Max das Haus streichen helfen
2 *dass wir Max das Haus helfen streichen


### Wählen Sie im Folgenden aus der Liste jeweils den Satz mit der entsprechenden syntaktische Struktur:


--- 
### a)  Satz mit *center embedding*-Struktur, d.h. die Verarbeitung benötigt mindestens eine CFG (ist also nicht mit einer regulären Grammatik möglich).

In [6]:
constructions[1]
#center embedding

'dass wir Max das Haus streichen helfen'

--- 

### b) Satz mit nicht-projektiver Struktur (*long distance dependency*), d.h. die Verarbeitung ist nur mit einem nicht-projektiven Dependency-Parser möglich.

In [7]:
constructions[2]
#non-projective

'*dass wir Max das Haus helfen streichen'

---
### c) Satz enthält *cross-serial-dependency*-Konstruktion, die nicht mit einer kontextfreien Grammatik beschrieben werden kann.

In [9]:
constructions[2]

'*dass wir Max das Haus helfen streichen'

---

---

 ### Erläuterung:

In [ ]:
#ERLAUTERUNG DER KONSTRUKTIONEN:
constructions = [
"dass wir Max_1 helfen_1 das Haus_2 zu streichen_2" #-
"dass wir Max_1 das Haus_2 streichen_2 helfen_1" #center embedding
"*dass wir Max_1 das Haus_2 helfen_1 streichen_2" #cross serial dependency (+non-projective)
]

In [57]:
#cross-serial-dependency-Satz mit DG:
sent_nr = """
1 *wir 4 nsubj
2 Max 4 iobj
3 Haus 5 obj
4 helfen 0 ROOT 
5 streichen 4 ccomp
"""

sent = transform_nr_conll(sent_nr)
dg = DependencyGraph(sent)

tree_labeled = dg.tree_labeled()
tree_labeled.pretty_print(unicodelines=True)  

ex = displacy_dep_input(sent)
html = displacy.render(ex, style="dep", manual=True, options={'distance':100})



            helfen(ROOT)                 
     ┌───────────┼──────────────┐         
     │           │       streichen(ccomp)
     │           │              │         
*wir(nsubj)  Max(iobj)      Haus(obj)    



In [34]:
#vgl. ProjectiveParser: nur Analyse für projektive Struktur (ohne long distance dependency):

grammar = nltk.DependencyGrammar.fromstring("""
    'helfen' ->  '*wir' | 'wir' | 'Max'  | 'streichen'
    'streichen' -> 'Haus'
    """)

parser = nltk.ProjectiveDependencyParser(grammar)

In [35]:
sentence = "wir Max Haus streichen helfen"    #projektiv    
for tree in parser.parse(sentence.split()):
    print(tree, "\n")
    tree.pretty_print(unicodelines=True)

(helfen wir Max (streichen Haus)) 

    helfen          
 ┌────┼────────┐     
 │    │    streichen
 │    │        │     
wir  Max      Haus  



In [36]:
sentence = "*wir Max Haus helfen streichen"  #nicht-projektiv
for tree in parser.parse(sentence.split()):
    print(tree, "\n")
    tree.pretty_print(unicodelines=True)

--- 


In [39]:
#CFG nur mit GAP-Feature möglich (Movement von V aus VP = cross serial dep.): 

sentence = "*wir Max Haus helfen streichen"

gramstring = r"""
% start S

    S -> NP VP
                    #GAP-INTRODUCTION:
    VP -> NP VP/V V V
                    #HERUNTERREICHEN GAP-INFORMATION:
    VP/?x -> NP V/?x
    NP -> N
                    #GAP-REALISIERUNG:    
    V/V -> 

    N -> '*wir' | 'Max' | 'Haus'
    V ->'streichen' | 'helfen'

"""

grammar = nltk.grammar.FeatureGrammar.fromstring(gramstring)
parser = nltk.parse.FeatureChartParser(grammar,trace=0)

for tree in parser.parse(sentence.split()):
    tree = Tree.fromstring(str(tree).replace(", ",","))
    tree.pretty_print(unicodelines=True)
    #display(tree)

          S[]                                   
 ┌─────────┴───────────────┐                     
 │                        VP[]                  
 │    ┌───────────┬────────┴──────┬────────┐     
 │    │        VP[]/V[]           │        │    
 │    │    ┌──────┴────────┐      │        │     
NP[] NP[] NP[]             │      │        │    
 │    │    │               │      │        │     
N[]  N[]  N[]           V[]/V[]  V[]      V[]   
 │    │    │               │      │        │     
*wir Max  Haus            ...   helfen streichen



--- 

In [19]:
#center-embedding-Satz mit CFG ist dagegen projektiv:

sentence = "wir Max Haus streichen helfen"

grammar = nltk.CFG.fromstring("""
    S -> NP VP
    VP -> NP VP V
    VP -> NP V
    NP -> N


    N -> 'wir' | 'Max' | 'Haus'
    V ->'streichen' | 'helfen'
""")

parser = nltk.ChartParser(grammar,trace=0)

for tree in parser.parse(sentence.split()):
    tree.pretty_print(unicodelines=True)    

     S                           
 ┌───┴───┐                        
 │       VP                      
 │   ┌───┴────┬──────────────┐    
 │   │        VP             │   
 │   │   ┌────┴──────┐       │    
 NP  NP  NP          │       │   
 │   │   │           │       │    
 N   N   N           V       V   
 │   │   │           │       │    
wir Max Haus     streichen helfen



In [61]:
#center-embedding-Satz mit DG:
sent_nr = """
1 wir 5 nsubj
2 Max 5 iobj
3 Haus 4 obj
4 streichen 5 ccomp
5 helfen 0 ROOT
"""

sent = transform_nr_conll(sent_nr)
dg = DependencyGraph(sent)

tree_labeled = dg.tree_labeled()
tree_labeled.pretty_print(unicodelines=True)  

ex = displacy_dep_input(sent)
html = displacy.render(ex, style="dep", manual=True, options={'distance':100})

           helfen(ROOT)                 
    ┌───────────┼──────────────┐         
    │           │       streichen(ccomp)
    │           │              │         
wir(nsubj)  Max(iobj)      Haus(obj)    



In [22]:
#zum Vergleich: erweiterter center-embedding-Satz

sentence = "wir den Max dem Hans das Haus streichen helfen lassen"

grammar = nltk.CFG.fromstring("""
    S -> NP VP
    VP -> NP VP V
    VP -> NP V
    VP -> V
    NP -> N
    NP -> Det N

    Det -> 'den' | 'dem' | 'das'
    N -> 'wir' | 'Max' | 'Haus' | 'Hans'
    V ->'streichen' | 'helfen' | 'lassen'
""")

parser = nltk.ChartParser(grammar,trace=0)

for tree in parser.parse(sentence.split()):
    tree.pretty_print(unicodelines=True)    

                         S                                       
 ┌───────────────────────┴────┐                                   
 │                            VP                                 
 │       ┌────────────────────┼──────────────────────────────┐    
 │       │                    VP                             │   
 │       │           ┌────────┴───────┬───────────────┐      │    
 │       │           │                VP              │      │   
 │       │           │            ┌───┴───────┐       │      │    
 NP      NP          NP           NP          │       │      │   
 │   ┌───┴───┐   ┌───┴───┐    ┌───┴───┐       │       │      │    
 N  Det      N  Det      N   Det      N       V       V      V   
 │   │       │   │       │    │       │       │       │      │    
wir den     Max dem     Hans das     Haus streichen helfen lassen

